# Stac_vrt for S3
> In this post. We try two things.   
First, we try use Stac_vrt to build Virtual tiff on S3 datasets.  
Second, we try array on S3 data [Image Array](https://developers.google.com/earth-engine/guides/arrays_array_images)

- toc: true 
- badges: true
- comments: true
- hide: true
- categories: [jupyter,Remote Sensing, Cloud Computing, Deep Learning]
- image: images/sentinel-2.png

In [1]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from pystac_client import Client

In [6]:
lon, lat = -105.78, 35.79

In [7]:
URL = "https://earth-search.aws.element84.com/v0"
catalog = Client.open(URL)

In [68]:
area={"type":"Polygon","coordinates":[[[111.137695,20.179724],[108.566895,20.179724],[108.566895,18.145852],[111.137695,18.145852],[111.137695,20.179724]]]}

In [71]:
%%time
items = catalog.search(
    intersects=area,
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-04-01/2020-05-31"
).get_all_items()
len(items)

CPU times: user 124 ms, sys: 8 ms, total: 132 ms
Wall time: 4.49 s


313

In [95]:
import stackstac
%time stack = stackstac.stack(items,epsg=3857)

CPU times: user 328 ms, sys: 0 ns, total: 328 ms
Wall time: 325 ms


In [96]:
stack

,Array,Chunk
Bytes,54.18 TiB,8.00 MiB
Shape,"(313, 17, 31489, 44444)","(1, 1, 1024, 1024)"
Count,7269850 Tasks,7257844 Chunks
Type,float64,numpy.ndarray


In [97]:
rgb = stack.sel(band=["B02"])

In [98]:
monthly = rgb.resample(time="MS").median("time", keep_attrs=True)

In [ ]:
import dask.diagnostics
with dask.diagnostics.ProgressBar():
    data = monthly.compute()